In [ ]:
import random
import math
import numpy as np

class AntColonyTSP:
    def __init__(self, cities, n_ants=50, n_iterations=1000, alpha=1, beta=5, rho=0.1, Q=100):
        self.cities = cities
        self.n_ants = n_ants
        self.n_iterations = n_iterations
        self.alpha = alpha
        self.beta = beta
        self.rho = rho
        self.Q = Q
        self.n_cities = len(cities)

        # Initialize pheromone matrix (initially with small values)
        self.pheromone = np.ones((self.n_cities, self.n_cities)) / self.n_cities
        # Initialize distance matrix (Euclidean distances)
        self.distance_matrix = self.calculate_distance_matrix()

    def calculate_distance_matrix(self):
        """ Calculate Euclidean distance matrix between cities. """
        dist_matrix = np.zeros((self.n_cities, self.n_cities))
        for i in range(self.n_cities):
            for j in range(i + 1, self.n_cities):
                dist = math.sqrt((self.cities[i][0] - self.cities[j][0]) ** 2 +
                                 (self.cities[i][1] - self.cities[j][1]) ** 2)
                dist_matrix[i][j] = dist_matrix[j][i] = dist
        return dist_matrix

    def choose_next_city(self, ant, visited):
        """ Probabilistically choose the next city based on pheromone and distance. """
        current_city = ant[-1]
        probabilities = []
        total_pheromone = 0.0

        for city in range(self.n_cities):
            if city not in visited:
                pheromone = self.pheromone[current_city][city] ** self.alpha
                distance = self.distance_matrix[current_city][city] ** self.beta
                total_pheromone += pheromone * (1 / distance)

        for city in range(self.n_cities):
            if city not in visited:
                pheromone = self.pheromone[current_city][city] ** self.alpha
                distance = self.distance_matrix[current_city][city] ** self.beta
                probability = (pheromone * (1 / distance)) / total_pheromone
                probabilities.append((city, probability))

        # Choose the next city based on the computed probabilities
        next_city = self.select_city(probabilities)
        return next_city

    def select_city(self, probabilities):
        """ Select city based on roulette wheel selection (probabilities). """
        rand = random.random()
        cumulative_prob = 0.0
        for city, prob in probabilities:
            cumulative_prob += prob
            if cumulative_prob >= rand:
                return city
        return probabilities[-1][0]  # If all else fails, return last option

    def construct_solution(self):
        """ Construct a solution by simulating an ant's tour. """
        solution = []
        visited = set()
        start_city = random.randint(0, self.n_cities - 1)
        solution.append(start_city)
        visited.add(start_city)

        while len(solution) < self.n_cities:
            next_city = self.choose_next_city(solution, visited)
            solution.append(next_city)
            visited.add(next_city)

        # Return to the starting city to complete the cycle
        solution.append(solution[0])
        return solution

    def calculate_solution_length(self, solution):
        """ Calculate the total distance of a given solution (tour). """
        length = 0.0
        for i in range(len(solution) - 1):
            length += self.distance_matrix[solution[i]][solution[i + 1]]
        return length

    def update_pheromones(self, ants):
        """ Update pheromone levels based on the ants' solutions. """
        # Pheromone evaporation
        self.pheromone *= (1 - self.rho)

        for ant in ants:
            solution_length = self.calculate_solution_length(ant)
            pheromone_deposit = self.Q / solution_length

            for i in range(len(ant) - 1):
                self.pheromone[ant[i]][ant[i + 1]] += pheromone_deposit

    def solve(self):
        best_solution = None
        best_length = float('inf')

        for iteration in range(self.n_iterations):
            ants = [self.construct_solution() for _ in range(self.n_ants)]
            for ant in ants:
                solution_length = self.calculate_solution_length(ant)
                if solution_length < best_length:
                    best_solution = ant
                    best_length = solution_length

            # Update pheromones after all ants have constructed their solutions
            self.update_pheromones(ants)

            # Optional: Print progress every 100 iterations
            if (iteration + 1) % 100 == 0:
                print(f"Iteration {iteration + 1}: Best Length = {best_length:.2f}")

        return best_solution, best_length

# Example usage
cities = [
    (0, 0),  # City 0
    (1, 3),  # City 1
    (4, 3),  # City 2
    (6, 1),  # City 3
    (5, 0)   # City 4
]

# Create an AntColonyTSP instance and solve the TSP
aco = AntColonyTSP(cities, n_ants=20, n_iterations=500, alpha=1, beta=2, rho=0.1, Q=100)
best_solution, best_length = aco.solve()

print("Best Solution:", best_solution)
print("Best Length:", best_length)


Iteration 100: Best Length = 15.40
Iteration 200: Best Length = 15.40
Iteration 300: Best Length = 15.40
Iteration 400: Best Length = 15.40
Iteration 500: Best Length = 15.40
Best Solution: [1, 2, 3, 4, 0, 1]
Best Length: 15.404918347287664


In [ ]:
import numpy as np
import random
import math

# Function to calculate the distance between two cities
def calculate_distance(city1, city2):
    return math.sqrt((city1[0] - city2[0]) ** 2 + (city1[1] - city2[1]) ** 2)

# Function to generate a distance matrix for the cities
def create_distance_matrix(cities):
    n = len(cities)
    distance_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            distance_matrix[i][j] = calculate_distance(cities[i], cities[j])
    return distance_matrix

# Ant Colony Optimization (ACO) Algorithm
def ant_colony_optimization(cities, n_ants, n_iterations, alpha=1, beta=5, rho=0.1, Q=100):
    # Number of cities
    n = len(cities)

    # Initialize pheromone matrix with small values
    pheromone_matrix = np.ones((n, n)) * 1e-6

    # Create distance matrix
    distance_matrix = create_distance_matrix(cities)

    # Best tour found
    best_tour = None
    best_tour_length = float('inf')

    # Main loop
    for iteration in range(n_iterations):
        # Initialize ants
        all_ants_tours = []
        all_ants_lengths = []

        # Loop through each ant
        for ant in range(n_ants):
            tour = []
            visited = [False] * n
            current_city = random.randint(0, n - 1)  # Start at a random city
            visited[current_city] = True
            tour.append(current_city)

            # Construct the solution (tour) for each ant
            for _ in range(n - 1):
                # Calculate probabilities of visiting each city
                probabilities = []
                for city in range(n):
                    if not visited[city]:
                        pheromone = pheromone_matrix[current_city][city] ** alpha
                        heuristic = (1.0 / distance_matrix[current_city][city]) ** beta
                        probabilities.append(pheromone * heuristic)
                    else:
                        probabilities.append(0)

                # Normalize the probabilities
                total = sum(probabilities)
                probabilities = [prob / total for prob in probabilities]

                # Choose the next city based on the probabilities (roulette wheel selection)
                next_city = np.random.choice(range(n), p=probabilities)
                tour.append(next_city)
                visited[next_city] = True
                current_city = next_city

            # Calculate the length of the tour
            tour_length = 0
            for i in range(n):
                tour_length += distance_matrix[tour[i]][tour[(i + 1) % n]]

            # Update the best tour if found a shorter one
            if tour_length < best_tour_length:
                best_tour_length = tour_length
                best_tour = tour

            # Store the ant's tour and its length
            all_ants_tours.append(tour)
            all_ants_lengths.append(tour_length)

        # Update pheromone matrix
        pheromone_matrix *= (1 - rho)  # Evaporate pheromone

        # Deposit pheromone for each ant's tour
        for ant in range(n_ants):
            pheromone_deposit = Q / all_ants_lengths[ant]
            for i in range(n):
                pheromone_matrix[all_ants_tours[ant][i]][all_ants_tours[ant][(i + 1) % n]] += pheromone_deposit

        print(f"Iteration {iteration + 1}/{n_iterations}, Best Length: {best_tour_length}")

    return best_tour, best_tour_length

# Example usage
if __name__ == "__main__":
    # Define the cities (x, y coordinates)
    cities = [
        (0, 0),  # City 0
        (1, 3),  # City 1
        (3, 1),  # City 2
        (6, 4),  # City 3
        (8, 0),  # City 4
    ]

    # Parameters
    n_ants = 10
    n_iterations = 20

    # Run the ACO algorithm
    best_tour, best_tour_length = ant_colony_optimization(cities, n_ants, n_iterations)

    # Output the best tour and its length
    print(f"Best Tour: {best_tour}")
    print(f"Best Tour Length: {best_tour_length}")


Iteration 1/20, Best Length: 22.655105068319532
Iteration 2/20, Best Length: 20.99473030252191
Iteration 3/20, Best Length: 20.99473030252191
Iteration 4/20, Best Length: 20.99473030252191
Iteration 5/20, Best Length: 20.99473030252191
Iteration 6/20, Best Length: 20.99473030252191
Iteration 7/20, Best Length: 20.99473030252191
Iteration 8/20, Best Length: 20.99473030252191
Iteration 9/20, Best Length: 20.99473030252191
Iteration 10/20, Best Length: 20.99473030252191
Iteration 11/20, Best Length: 20.99473030252191
Iteration 12/20, Best Length: 20.99473030252191
Iteration 13/20, Best Length: 20.99473030252191
Iteration 14/20, Best Length: 20.99473030252191
Iteration 15/20, Best Length: 20.99473030252191
Iteration 16/20, Best Length: 20.99473030252191
Iteration 17/20, Best Length: 20.99473030252191
Iteration 18/20, Best Length: 20.99473030252191
Iteration 19/20, Best Length: 20.99473030252191
Iteration 20/20, Best Length: 20.99473030252191
Best Tour: [4, 3, 1, 0, 2]
Best Tour Length: 20.